## Part1: Scraping from GoFundMe for Animal Rescue Campaigns 

In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 29.7 MB/s eta 0:00:00


In [ ]:

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup
import re
from datetime import datetime
import json
import os

#### Task A : Scrapper to pull 1000 campaigns

In [ ]:

# class GoFundMeScraper:

#     def __init__(self, category_url, target_count=1000, verify_category=True):
#         self.category_url = category_url
#         self.target_count = target_count
#         self.verify_category = verify_category
#         self.campaigns_data = []
#         self.driver = None

#         # Determine expected category from URL
#         self.expected_category = None
#         if "animal" in category_url.lower():  # Matches both "animal" and "animals"
#             self.expected_category = "animals"
#         elif "medical-fundraiser" in category_url.lower():
#             self.expected_category = "medical"
#         elif "/s?q=" in category_url:
#             self.expected_category = "search"

#     def setup_driver(self):
#         from selenium.webdriver.chrome.options import Options
#         from selenium.webdriver.chrome.service import Service
#         from webdriver_manager.chrome import ChromeDriverManager
#         import subprocess
#         import os
#         import shutil

#         options = Options()
#         options.add_argument('--no-sandbox')
#         options.add_argument('--disable-dev-shm-usage')
#         options.add_argument('--disable-blink-features=AutomationControlled')
#         options.add_argument('--disable-gpu')  # Added for Mac compatibility
#         options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36')

#         try:
#             # getting the chromdriver path
#             driver_path = ChromeDriverManager().install()
#             print(f"ChromeDriver path: {driver_path}")

#             # this is specific for mac as we I'm suing mac this may be diffferent for windows
#             if 'chromedriver-mac-arm64' in driver_path:
#                 # The actual binary might be inside a subfolder
#                 parent_dir = os.path.dirname(driver_path)
#                 actual_binary = os.path.join(parent_dir, 'chromedriver')


#                 if os.path.exists(actual_binary):
#                     driver_path = actual_binary
#                     print(f"Using binary at: {driver_path}")


#                 os.chmod(driver_path, 0o755)

#                 # trying to remove quarantine
#                 try:
#                     subprocess.run(['xattr', '-d', 'com.apple.quarantine', driver_path],
#                                  stderr=subprocess.DEVNULL, check=False)
#                 except:
#                     pass

#                 # Try to code sign it
#                 try:
#                     subprocess.run(['codesign', '--force', '--deep', '--sign', '-', driver_path],
#                                  stderr=subprocess.DEVNULL, check=False)
#                     print("✓ Code signed the driver")
#                 except:
#                     pass

#             service = Service(driver_path)
#             self.driver = webdriver.Chrome(service=service, options=options)
#             self.driver.maximize_window()

#             print("✓ WebDriver initialized successfully")

#         except Exception as e:
#             print(f"\n✗ Error: {e}")
#             print("\n" + "="*60)
#             print("MANUAL FIX REQUIRED - Run this in Terminal:")
#             print("="*60)
#             print(f"sudo codesign --force --deep --sign - {driver_path}")
#             print("\nOr try:")
#             print(f"chmod +x {driver_path}")
#             print("\nThen run the script again.")
#             print("="*60)
#             raise

#     def scroll_page(self, scroll_pause_time=2, max_scrolls=50):
#         last_height = self.driver.execute_script("return document.body.scrollHeight")
#         scrolls = 0

#         while scrolls < max_scrolls:
#             self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#             time.sleep(scroll_pause_time)
#             new_height = self.driver.execute_script("return document.body.scrollHeight")

#             if new_height == last_height:
#                 print(f"Reached end of page after {scrolls} scrolls")
#                 break

#             last_height = new_height
#             scrolls += 1

#             if scrolls % 10 == 0:
#                 print(f"Scrolled {scrolls} times, loading more campaigns...")

#     def extract_campaign_links(self):
#         soup = BeautifulSoup(self.driver.page_source, 'html.parser')
#         links = []

#         selectors = ['a[href*="/f/"]', 'a[class*="campaign"]', 'a[class*="fundraiser"]']

#         for selector in selectors:
#             elements = soup.select(selector)
#             for elem in elements:
#                 href = elem.get('href', '')
#                 if '/f/' in href:
#                     full_url = href if href.startswith('http') else f"https://www.gofundme.com{href}"
#                     if full_url not in links:
#                         links.append(full_url)

#         print(f"✓ Found {len(links)} unique campaign links")

#         # getting some smaple campaigns to check if that works fine
#         if links and len(links) > 0:
#             print(f"\n Sample campaign URLs:")
#             for i, url in enumerate(links[:5], 1):
#                 campaign_slug = url.split('/f/')[-1].split('?')[0]
#                 print(f"   {i}. .../{campaign_slug}")

#         return links

#     def parse_duration(self, duration_text):
#         if not duration_text:
#             return None

#         duration_text = str(duration_text).strip()
#         duration_lower = duration_text.lower()

#         # using regular expressions to pull the dates as there are different patterns "X days ago", "X months ago", "X years ago"
#         if 'ago' in duration_lower:
#             # handling both full and abbreviated forms: "6 days ago" or "6 d ago" or "6d ago"
#             patterns = [
#                 (r'(\d+)\s*(?:days?|d)\s*ago', 1),           # "6 days ago" or "6 d ago"
#                 (r'(\d+)\s*(?:months?|mo?)\s*ago', 30),      # "6 months ago" or "6 m ago"
#                 (r'(\d+)\s*(?:years?|y)\s*ago', 365),        # "6 years ago" or "6 y ago"
#                 (r'(\d+)\s*(?:weeks?|w)\s*ago', 7),          # "6 weeks ago" or "6 w ago"
#                 (r'(\d+)\s*(?:hours?|h)\s*ago', 0),          # "6 hours ago" = 0 days (same day)
#             ]

#             for pattern, multiplier in patterns:
#                 match = re.search(pattern, duration_lower)
#                 if match:
#                     num = int(match.group(1))
#                     return max(0, num * multiplier)  # Ensure non-negative

#         #  to parse as a date
#         # removing the  common prefixes
#         date_text = re.sub(r'created|launched|started|on', '', duration_lower, flags=re.IGNORECASE).strip()
#         date_text = duration_text  # Use original case for month names

#         # various date formats toc casting
#         date_formats = [
#             '%d %B %Y',      # "24 October 2025"
#             '%B %d, %Y',     # "October 24, 2025"
#             '%B %Y',         # "October 2025"
#             '%d %b %Y',      # "24 Oct 2025"
#             '%b %d, %Y',     # "Oct 24, 2025"
#             '%b %Y',         # "Oct 2025"
#             '%Y-%m-%d',      # "2025-10-24" (ISO format)
#             '%m/%d/%Y',      # "10/24/2025"
#             '%d/%m/%Y',      # "24/10/2025"
#         ]

#         for fmt in date_formats:
#             try:
#                 # Extract just the date part
#                 date_match = re.search(r'\d{1,2}[\s/\-]\w+[\s/\-]\d{4}|\w+[\s/\-]\d{1,2}[\s/\-]\d{4}|\w+[\s/\-]\d{4}|\d{4}[\-/]\d{2}[\-/]\d{2}|\d{1,2}[\-/]\d{1,2}[\-/]\d{4}', date_text)
#                 if date_match:
#                     date_str = date_match.group()
#                     created_date = datetime.strptime(date_str, fmt)
#                     days_running = (datetime.now() - created_date).days
#                     return max(0, days_running)
#             except ValueError:
#                 continue

#         return None

#     def parse_amount_raised(self, amount_text):
#         if not amount_text:
#             return 0.0

#         amount_text = amount_text.replace(',', '').replace('$', '').lower()

#         multiplier = 1
#         if 'k' in amount_text:
#             multiplier = 1000
#             amount_text = amount_text.replace('k', '')
#         elif 'm' in amount_text:
#             multiplier = 1000000
#             amount_text = amount_text.replace('m', '')

#         match = re.search(r'([\d.]+)', amount_text)
#         if match:
#             return float(match.group(1)) * multiplier

#         return 0.0

#     def verify_campaign_category(self, soup, all_text):
#         """
#         Verify if campaign belongs to the expected category
#         Returns: (is_valid, detected_category)
#         """
#         if not self.verify_category or not self.expected_category:
#             return True, "not_verified"

#         detected_category = "unknown"

#         # looking for category indicators in the page
#         category_keywords = {
#             'animals': ['pet', 'dog', 'cat', 'animal', 'rescue', 'shelter', 'veterinary', 'vet',
#                        'puppy', 'kitten', 'horse', 'wildlife', 'bird', 'rabbit', 'hamster',
#                        'guinea pig', 'adoption', 'spay', 'neuter', 'stray', 'sanctuary'],
#             'medical': ['medical', 'health', 'hospital', 'treatment', 'surgery', 'cancer',
#                        'diagnosis', 'therapy', 'medication', 'doctor', 'chemotherapy', 'disease',
#                        'illness', 'patient', 'emergency room', 'ICU', 'transplant'],
#             'memorial': ['memorial', 'funeral', 'burial', 'passed away', 'in memory',
#                         'celebration of life', 'rest in peace', 'obituary'],
#             'emergency': ['emergency', 'disaster', 'fire', 'flood', 'hurricane', 'accident',
#                          'urgent', 'crisis']
#         }

#         # was able to get some random campaigns from different categories so using some keywords eliminated those campaigns
#         human_medical_indicators = [
#             'cancer treatment', 'chemotherapy', 'radiation therapy', 'tumor', 'diagnosis',
#             'patient', 'hospital bills', 'medical bills', 'surgery for', 'treatment for',
#             'my battle', 'his battle', 'her battle', 'fighting cancer', 'beating cancer'
#         ]

#         text_lower = all_text.lower()


#         if self.expected_category == 'animals':
#             for indicator in human_medical_indicators:
#                 if indicator in text_lower:
#                     # Check if it's about a person (not a pet)
#                     # Look for context around the medical term
#                     context_words = ['my', 'his', 'her', 'their', 'husband', 'wife', 'son',
#                                    'daughter', 'father', 'mother', 'brother', 'sister',
#                                    'friend', 'family member', 'loved one', 'person', 'human']

#                     # Check proximity of medical term to human indicators
#                     for context in context_words:
#                         if context in text_lower:
#                             # Strong indication this is human medical, not animal
#                             return False, 'medical_human'


#         meta_category = soup.find('meta', property='fundraiser:category')
#         if meta_category:
#             cat_value = meta_category.get('content', '').lower()
#             if 'animal' in cat_value or 'pet' in cat_value:
#                 return True, 'animals'
#             elif 'medical' in cat_value and self.expected_category == 'animals':
#                 return False, 'medical'


#         category_scores = {}
#         for cat, keywords in category_keywords.items():
#             score = sum(1 for keyword in keywords if keyword in text_lower)
#             category_scores[cat] = score

#         # determine category based on highest score
#         if category_scores:
#             detected_category = max(category_scores, key=category_scores.get)
#             max_score = category_scores[detected_category]
#             animal_score = category_scores.get('animals', 0)
#             medical_score = category_scores.get('medical', 0)

#             # if the expected category is animals
#             if self.expected_category == 'animals':
#                 # Must have strong animal indicators
#                 if animal_score >= 3:
#                     # Good animal campaign
#                     return True, 'animals'
#                 elif medical_score > animal_score:
#                     # More medical than animal - likely human medical
#                     return False, 'medical'
#                 elif animal_score < 2 and max_score > 0:
#                     # Not enough animal evidence
#                     return False, detected_category
#                 else:
#                     # Uncertain - be conservative and skip
#                     return False, 'uncertain'

#         #  search queries, be more lenient
#         if self.expected_category == 'search':
#             return True, detected_category

#         # default set: if no clear category match, skip it for animals
#         if self.expected_category == 'animals':
#             return False, 'insufficient_evidence'

#         return True, detected_category

#     def scrape_campaign_details(self, campaign_url):
#         try:
#             self.driver.get(campaign_url)


#             time.sleep(5)

#             # loading adn waiting for long time for dynamic data
#             try:
#                 WebDriverWait(self.driver, 20).until(
#                     EC.presence_of_element_located((By.TAG_NAME, "h1"))
#                 )
#             except TimeoutException:
#                 print(f"Timeout loading: {campaign_url}")
#                 return None

#             # loading more campaigns using show-more buttion on the website
#             self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
#             time.sleep(2)
#             self.driver.execute_script("window.scrollTo(0, 0);")
#             time.sleep(1)

#             page_source = self.driver.page_source
#             soup = BeautifulSoup(page_source, 'html.parser')

#             data = {
#                 'campaign_url': campaign_url,
#                 'image_url': None,
#                 'image_urls': [],  # NEW: Store all images
#                 'description': None,
#                 'amount_raised': 0.0,
#                 'campaign_duration_days': None
#             }

#             # pulling all image URLs from the campaign
#             image_urls_found = []

#             # pullin in the primary image
#             meta_image = soup.find('meta', property='og:image')
#             if meta_image and meta_image.get('content'):
#                 img_url = meta_image.get('content')
#                 if img_url not in image_urls_found:
#                     image_urls_found.append(img_url)

#             # pulling all other images
#             for img in soup.find_all('img'):
#                 src = img.get('src', '') or img.get('data-src', '') or img.get('data-lazy-src', '')
#                 # Filter for actual campaign images (not icons, logos, etc.)
#                 if src and len(src) > 30:  # Substantial URLs only
#                     # Skip common non-campaign images
#                     skip_keywords = ['logo', 'icon', 'avatar', 'badge', 'flag', 'share', 'facebook', 'twitter']
#                     if not any(keyword in src.lower() for keyword in skip_keywords):
#                         # Make sure it's a full URL
#                         if src.startswith('http'):
#                             full_url = src
#                         elif src.startswith('//'):
#                             full_url = 'https:' + src
#                         elif src.startswith('/'):
#                             full_url = 'https://www.gofundme.com' + src
#                         else:
#                             continue

#                         if full_url not in image_urls_found:
#                             image_urls_found.append(full_url)

#             # storing the primary and all other images
#             data['image_urls'] = image_urls_found
#             data['image_url'] = image_urls_found[0] if image_urls_found else None

#             print(f"   → Found {len(image_urls_found)} images")

#             # getting the descriptions
#             meta_desc = soup.find('meta', property='og:description')
#             if meta_desc and meta_desc.get('content'):
#                 desc_text = meta_desc.get('content')
#                 # Only use if it's substantial (not just default text)
#                 if len(desc_text) > 50 and 'gofundme' not in desc_text.lower():
#                     data['description'] = desc_text


#             if not data['description']:
#                 # Look for text blocks that are likely the story
#                 all_text_blocks = []
#                 for div in soup.find_all(['div', 'p']):
#                     text = div.get_text(strip=True)
#                     if 100 < len(text) < 5000:  # Reasonable description length
#                         all_text_blocks.append(text)


#                 if all_text_blocks:
#                     data['description'] = max(all_text_blocks, key=len)


#             amount_found = False


#             try:
#                 possible_amounts = []

#                 all_text = self.driver.find_element(By.TAG_NAME, 'body').text

#                 # patterns for amounts like "£17,218 raised" or "$5,234 raised"
#                 amount_patterns = [
#                     r'[£$€][\d,]+(?:\.\d{2})?\s*raised',
#                     r'[£$€][\d,]+(?:\.\d{2})?\s*of',
#                     r'raised\s*[£$€][\d,]+(?:\.\d{2})?'
#                 ]

#                 for pattern in amount_patterns:
#                     matches = re.findall(pattern, all_text, re.IGNORECASE)
#                     if matches:
#                         for match in matches:
#                             # Extract just the number
#                             num_match = re.search(r'[\d,]+(?:\.\d{2})?', match)
#                             if num_match:
#                                 amount = self.parse_amount_raised(num_match.group())
#                                 if amount > 0:
#                                     possible_amounts.append(amount)

#                 # pulling the first reasonable amount found
#                 if possible_amounts:
#                     # Filter out unrealistic values
#                     realistic_amounts = [a for a in possible_amounts if 0 < a < 100000000]
#                     if realistic_amounts:
#                         data['amount_raised'] = realistic_amounts[0]
#                         amount_found = True

#             except Exception as e:
#                 print(f"   Error extracting amount: {e}")

#             if not amount_found:
#                 meta_amount = soup.find('meta', property='fundraiser:amount_raised')
#                 if meta_amount and meta_amount.get('content'):
#                     data['amount_raised'] = self.parse_amount_raised(meta_amount.get('content'))
#                     amount_found = True

#             # getting campaign duration from visible text
#             duration_found = False

#             try:
#                 all_text = self.driver.find_element(By.TAG_NAME, 'body').text

#                 if 'day' in all_text.lower() or 'created' in all_text.lower():
#                     # Extract lines containing date/duration info
#                     lines = all_text.split('\n')
#                     date_lines = [line for line in lines if 'day' in line.lower() or 'created' in line.lower() or 'month' in line.lower() or 'year' in line.lower()]
#                     if date_lines:
#                         print(f"   Found date text: {date_lines[:3]}")  # Debug: show what we found

#                 pattern1 = r'[Cc]reated\s+(\d+)\s*(?:days?|months?|years?|d|mo?|y)\s*ago'
#                 match = re.search(pattern1, all_text)
#                 if match:
#                     # Use the full text match for parsing
#                     full_match = match.group(0)
#                     days = self.parse_duration(full_match)
#                     if days is not None and days >= 0:
#                         data['campaign_duration_days'] = days
#                         duration_found = True
#                         print(f"   Matched pattern: {full_match} = {days} days")

#                 if not duration_found:
#                     pattern2 = r'(\d+)\s+(day|month|year)s?\s+(?:until|left|remaining)'
#                     match = re.search(pattern2, all_text, re.IGNORECASE)
#                     if match:
#                         num = int(match.group(1))
#                         unit = match.group(2).lower()

#                         if 'day' in unit:
#                             data['campaign_duration_days'] = num
#                         elif 'month' in unit:
#                             data['campaign_duration_days'] = num * 30
#                         elif 'year' in unit:
#                             data['campaign_duration_days'] = num * 365

#                         duration_found = True
#                         print(f"   Matched pattern: {num} {unit}s until/left")

#                 if not duration_found:
#                     # Look for month names
#                     month_pattern = r'(\d{1,2}\s+)?([Jj]anuary|[Ff]ebruary|[Mm]arch|[Aa]pril|[Mm]ay|[Jj]une|[Jj]uly|[Aa]ugust|[Ss]eptember|[Oo]ctober|[Nn]ovember|[Dd]ecember)\s+(\d{4})'
#                     matches = re.findall(month_pattern, all_text)

#                     for match in matches:
#                         day = match[0].strip() if match[0] else '1'
#                         month = match[1]
#                         year = match[2]

#                         date_str = f"{day} {month} {year}"
#                         days = self.parse_duration(f"Created {date_str}")
#                         if days is not None and days > 0:  # Make sure it's positive
#                             data['campaign_duration_days'] = days
#                             duration_found = True
#                             print(f"   Parsed date: {date_str} = {days} days")
#                             break

#                 if not duration_found:
#                     pattern4 = r'[Cc]reated\s+([Jj]anuary|[Ff]ebruary|[Mm]arch|[Aa]pril|[Mm]ay|[Jj]une|[Jj]uly|[Aa]ugust|[Ss]eptember|[Oo]ctober|[Nn]ovember|[Dd]ecember)\s+(\d{4})'
#                     match = re.search(pattern4, all_text)
#                     if match:
#                         month = match.group(1)
#                         year = match.group(2)

#                         date_str = f"{month} {year}"
#                         days = self.parse_duration(date_str)
#                         if days is not None and days >= 0:
#                             data['campaign_duration_days'] = days
#                             duration_found = True
#                             print(f"   Parsed date: {date_str} = {days} days")

#                 if not duration_found:
#                     # Try to find any text with "Created" followed by date info
#                     created_pattern = r'[Cc]reated[:\s]+([^\n]{5,50})'
#                     match = re.search(created_pattern, all_text)
#                     if match:
#                         date_text = match.group(1).strip()
#                         print(f"   Found 'Created' text: {date_text}")
#                         days = self.parse_duration(date_text)
#                         if days is not None and days >= 0:
#                             data['campaign_duration_days'] = days
#                             duration_found = True

#             except Exception as e:
#                 print(f"   Error extracting duration: {e}")

#             if not duration_found:
#                 for time_elem in soup.find_all('time'):
#                     datetime_attr = time_elem.get('datetime', '')
#                     text = time_elem.get_text(strip=True)

#                     if datetime_attr:
#                         days = self.parse_duration(datetime_attr)
#                         if days is not None and days >= 0:
#                             data['campaign_duration_days'] = days
#                             duration_found = True
#                             print(f"   Found in time element: {datetime_attr} = {days} days")
#                             break
#                     elif text:
#                         days = self.parse_duration(text)
#                         if days is not None and days >= 0:
#                             data['campaign_duration_days'] = days
#                             duration_found = True
#                             print(f"   Found in time text: {text} = {days} days")
#                             break

#             if not duration_found:
#                 print(f"   No duration found - campaign may be very recent")


#             print(f"   Image: {'Yes' if data['image_url'] else 'No'}")
#             print(f"   Description: {len(data['description']) if data['description'] else 0} chars")
#             print(f"   Amount: ${data['amount_raised']:,.2f}")
#             print(f"   Duration: {data['campaign_duration_days']} days")

#             # Show snippet of what we found for verification
#             if data['description']:
#                 snippet = data['description'][:80] + "..." if len(data['description']) > 80 else data['description']
#                 print(f"   → Desc preview: {snippet}")

#             return data

#         except Exception as e:
#             print(f" Error scraping {campaign_url}: {str(e)}")
#             import traceback
#             traceback.print_exc()
#             return None

#     def save_progress(self):
#         if self.campaigns_data:
#             df_temp = pd.DataFrame(self.campaigns_data)
#             df_temp.to_csv('gofundme_campaigns_progress.csv', index=False)
#             print(f"\n Progress saved: {len(df_temp)} campaigns")

#     def scrape(self):
#         print("="*60)
#         print("GoFundMe Campaign Scraper - Starting")
#         print("="*60)

#         self.setup_driver()


#         discover_url = "https://www.gofundme.com/discover"
#         print(f"\nStep 1: Loading GoFundMe Discover page...")
#         print(f"  URL: {discover_url}")
#         self.driver.get(discover_url)
#         time.sleep(5)  # Wait for page to load


#         print(f"\nStep 2: Looking for Animals category...")

#         try:
#             animal_link_found = False

#             possible_selectors = [
#                 "//a[contains(text(), 'Animal')]",
#                 "//a[contains(@href, 'animal-fundraiser')]",
#                 "//a[contains(@href, 'animals-fundraiser')]",
#             ]

#             for selector in possible_selectors:
#                 try:
#                     element = self.driver.find_element(By.XPATH, selector)
#                     element_text = element.text
#                     element_href = element.get_attribute('href')
#                     print(f"  ✓ Found: {element_text} -> {element_href}")

#                     self.driver.execute_script("arguments[0].scrollIntoView(true);", element)
#                     time.sleep(1)
#                     element.click()
#                     animal_link_found = True
#                     print(f"  ✓ Clicked on Animals category!")
#                     break
#                 except:
#                     continue

#             if not animal_link_found:
#                 print("  ✗ Could not find Animals category link")
#                 print("  → Trying direct URL instead...")
#                 self.driver.get("https://www.gofundme.com/discover/animal-fundraiser")

#             time.sleep(5)
#             print(f"  ✓ Animals category page loaded")
#             print(f"  Current URL: {self.driver.current_url}")

#         except Exception as e:
#             print(f"  ✗ Error finding Animals category: {e}")
#             print(f"  → Navigating to category URL directly...")
#             self.driver.get(self.category_url)
#             time.sleep(3)

#         print("\nStep 3: Loading campaigns by clicking 'Show more' button...")
#         print(f"  Need to load at least {self.target_count} campaign links...")

#         campaigns_loaded = 0
#         click_attempts = 0
#         max_clicks = 150
#         no_button_count = 0

#         while campaigns_loaded < self.target_count and click_attempts < max_clicks:

#             self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#             time.sleep(2)

#             soup_temp = BeautifulSoup(self.driver.page_source, 'html.parser')
#             links_temp = soup_temp.select('a[href*="/f/"]')
#             unique_links = set([l.get('href') for l in links_temp if l.get('href') and '/f/' in l.get('href')])
#             campaigns_loaded = len(unique_links)

#             button_clicked = False

#             try:
#                 buttons = self.driver.find_elements(By.TAG_NAME, "button")
#                 for button in buttons:
#                     button_text = button.text.lower().strip()
#                     if any(phrase in button_text for phrase in ['show more', 'load more', 'see more', 'view more']):
#                         print(f"  Found button: '{button.text}'")
#                         self.driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
#                         time.sleep(1)
#                         try:
#                             button.click()
#                         except:
#                             self.driver.execute_script("arguments[0].click();", button)
#                         button_clicked = True
#                         click_attempts += 1
#                         time.sleep(3)
#                         break
#             except:
#                 pass

#             if not button_clicked:
#                 try:
#                     clickable_elements = self.driver.find_elements(By.XPATH, "//*[contains(text(), 'more') or contains(text(), 'More')]")
#                     for elem in clickable_elements:
#                         if elem.is_displayed():
#                             elem_text = elem.text.lower().strip()
#                             if 'show' in elem_text or 'load' in elem_text or 'see' in elem_text:
#                                 print(f"  Found clickable: '{elem.text}'")
#                                 self.driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", elem)
#                                 time.sleep(1)
#                                 try:
#                                     elem.click()
#                                 except:
#                                     self.driver.execute_script("arguments[0].click();", elem)
#                                 button_clicked = True
#                                 click_attempts += 1
#                                 time.sleep(3)
#                                 break
#                 except:
#                     pass

#             if button_clicked:
#                 no_button_count = 0
#                 if click_attempts % 5 == 0:
#                     print(f"  Click #{click_attempts}: Loaded {campaigns_loaded} campaigns...")
#             else:
#                 no_button_count += 1
#                 if no_button_count == 1:
#                     print(f"  No 'Show more' button found")
#                     print(f"  Total campaigns visible: {campaigns_loaded}")

#                 # If button not found multiple times, probably reached the end
#                 if no_button_count >= 3:
#                     print(f"  Reached end of available campaigns")
#                     break

#             if campaigns_loaded >= self.target_count:
#                 print(f"  Loaded {campaigns_loaded} campaigns (target: {self.target_count})")
#                 break

#         print(f"  Finished loading: {click_attempts} button clicks, {campaigns_loaded} campaigns total")

#         print("\nStep 4: Extracting campaign links from Animals category...")
#         campaign_links = self.extract_campaign_links()

#         if not campaign_links:
#             print(" No campaign links found. Please check the page.")
#             self.driver.quit()
#             return pd.DataFrame()

#         campaign_links = campaign_links[:min(len(campaign_links), self.target_count)]
#         print(f"\nWill scrape {len(campaign_links)} campaigns (target: {self.target_count})")

#         print("\n" + "="*60)
#         print("Scraping individual campaigns...")
#         print("="*60)

#         for i, url in enumerate(campaign_links, 1):
#             print(f"\n[{i}/{len(campaign_links)}] Scraping: {url}")

#             campaign_data = self.scrape_campaign_details(url)

#             if campaign_data:
#                 self.campaigns_data.append(campaign_data)
#                 print(f"✓ Success")

#             if i % 50 == 0:
#                 self.save_progress()

#             time.sleep(1.5)

#         self.driver.quit()

#         df = pd.DataFrame(self.campaigns_data)

#         print("\n" + "="*60)
#         print("Scraping Complete!")
#         print("="*60)
#         print(f"Successfully scraped: {len(df)} campaigns")
#         print(f"\nData preview:")
#         print(df.head())

#         return df


# if __name__ == "__main__":

#     CATEGORY_URL = "https://www.gofundme.com/discover/animals-fundraiser"
#     TARGET_COUNT = 1000

#     scraper = GoFundMeScraper(
#         category_url=CATEGORY_URL,
#         target_count=TARGET_COUNT
#     )

#     df_campaigns = scraper.scrape()

#     if not df_campaigns.empty:
#         output_file = 'gofundme_campaigns_data.csv'
#         df_campaigns.to_csv(output_file, index=False)
#         print(f"\n Final data saved to: {output_file}")
#         print(f" TEAMMATES: Use '{output_file}' for Tasks B-F")

#         df_campaigns.to_json('gofundme_campaigns_data.json', orient='records', indent=2)

#         if os.path.exists('gofundme_campaigns_progress.csv'):
#             os.remove('gofundme_campaigns_progress.csv')
#             print(f" Cleaned up temporary progress file")

#         print("\n" + "="*60)
#         print("FINAL STATISTICS")
#         print("="*60)
#         print(f"Total campaigns scraped: {len(df_campaigns)}")
#         print(f"\nDATA COMPLETENESS (Assignment Requirements):")
#         print(f"  (i)   Image URLs (primary): {df_campaigns['image_url'].notna().sum()} / {len(df_campaigns)} ({df_campaigns['image_url'].notna().sum()/len(df_campaigns)*100:.1f}%)")
#         print(f"        Total images collected: {df_campaigns['image_urls'].apply(len).sum()} across all campaigns")
#         print(f"        Avg images per campaign: {df_campaigns['image_urls'].apply(len).mean():.1f}")
#         print(f"  (ii)  Descriptions: {df_campaigns['description'].notna().sum()} / {len(df_campaigns)} ({df_campaigns['description'].notna().sum()/len(df_campaigns)*100:.1f}%)")
#         print(f"  (iii) $ Raised: {(df_campaigns['amount_raised'] > 0).sum()} / {len(df_campaigns)} ({(df_campaigns['amount_raised'] > 0).sum()/len(df_campaigns)*100:.1f}%)")
#         print(f"  (iv)  Duration: {df_campaigns['campaign_duration_days'].notna().sum()} / {len(df_campaigns)} ({df_campaigns['campaign_duration_days'].notna().sum()/len(df_campaigns)*100:.1f}%)")
#         print(f"\nFUNDRAISING STATISTICS:")
#         print(f"  Average amount raised: ${df_campaigns['amount_raised'].mean():,.2f}")
#         print(f"  Median amount raised: ${df_campaigns['amount_raised'].median():,.2f}")
#         print(f"  Max amount raised: ${df_campaigns['amount_raised'].max():,.2f}")
#         print(f"  Min amount raised: ${df_campaigns['amount_raised'].min():,.2f}")
#         if df_campaigns['campaign_duration_days'].notna().sum() > 0:
#             print(f"\nCAMPAIGN DURATION STATISTICS:")
#             print(f"  Average duration: {df_campaigns['campaign_duration_days'].mean():.1f} days")
#             print(f"  Median duration: {df_campaigns['campaign_duration_days'].median():.1f} days")
#             print(f"  Max duration: {df_campaigns['campaign_duration_days'].max():.0f} days")
#             print(f"  Min duration: {df_campaigns['campaign_duration_days'].min():.0f} days")

#     else:
#         print("\n No data scraped. URL issue again.")

GoFundMe Campaign Scraper - Starting
ChromeDriver path: /Users/rohan/.wdm/drivers/chromedriver/mac64/141.0.7390.54/chromedriver-mac-arm64/chromedriver
Using binary at: /Users/rohan/.wdm/drivers/chromedriver/mac64/141.0.7390.54/chromedriver-mac-arm64/chromedriver
✓ Code signed the driver
✓ WebDriver initialized successfully

Step 1: Loading GoFundMe Discover page...
  URL: https://www.gofundme.com/discover

Step 2: Looking for Animals category...
  ✓ Found: Animal -> https://www.gofundme.com/discover/animal-fundraiser
  ✓ Clicked on Animals category!
  ✓ Animals category page loaded
  Current URL: https://www.gofundme.com/discover/animal-fundraiser

Step 3: Loading campaigns by clicking 'Show more' button...
  Need to load at least 1000 campaign links...
  Found button: 'Show more'
  Found button: 'Show more'
  Found button: 'Show more'
  Found button: 'Show more'
  Found button: 'Show more'
  Click #5: Loaded 60 campaigns...
  Found button: 'Show more'
  Found button: 'Show more'
  Fou

## Part 2: Obtaining Image Lables using Google Vision API 

In [1]:
# pip install google-cloud-vision

In [12]:
# # ----------------------------
# # GOOGLE VISION API IMAGE LABELING SCRIPT
# # ----------------------------

# import os
# import io
# import requests
# import pandas as pd
# from PIL import Image
# from google.cloud import vision
# from tqdm import tqdm

# # ----------------------------
# # CONFIGURATION
# # ----------------------------
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "vision-api-test-474617-b93a4a02e265.json"
# INPUT_CSV = "gofundme_campaigns_data.csv"
# OUTPUT_CSV = "gofundme_with_labels.csv"
# INTERMEDIATE_SAVE = 100  # save every N images

# # ----------------------------
# # INITIALIZE CLIENT
# # ----------------------------
# client = vision.ImageAnnotatorClient()

# # LOAD DATA
# df = pd.read_csv(INPUT_CSV)
# labels_list = []

# # PROCESS IMAGES
# for i, url in enumerate(tqdm(df["image_url"], desc="Processing Images"), 1):
#     try:
#         # 1) Download image
#         r = requests.get(url, timeout=15)
#         if r.status_code != 200:
#             labels_list.append("ERROR_DOWNLOAD")
#             print(f"[{i}/{len(df)}] Download error (status code {r.status_code})")
#             continue

#         # 2) Check size and compress if >20 MB
#         size_MB = len(r.content) / (1024 * 1024)
#         if size_MB > 20:
#             img = Image.open(io.BytesIO(r.content))
#             buf = io.BytesIO()
#             img.convert("RGB").save(buf, format="JPEG", quality=85)  # compress
#             image_bytes = buf.getvalue()
#         else:
#             image_bytes = r.content

#         # 3) Send to Vision API
#         image = vision.Image(content=image_bytes)
#         response = client.label_detection(image=image)

#         if response.error.message:
#             labels_list.append("ERROR_API")
#             print(f"[{i}/{len(df)}] Vision API error: {response.error.message}")
#             continue

#         # 4) Extract labels
#         labels = [label.description for label in response.label_annotations]
#         labels_list.append(", ".join(labels) if labels else "NO_LABELS")

#         # 5) Optional: show first 3 labels for progress
#         print(f"[{i}/{len(df)}] Labels: {labels[:3]}...")

#     except Exception as e:
#         labels_list.append("ERROR_EXCEPTION")
#         print(f"[{i}/{len(df)}] Exception: {e}")

#     # 6) Save intermediate results
#     if i % INTERMEDIATE_SAVE == 0:
#         df_temp = df.copy()
#         df_temp["image_labels"] = labels_list + [""] * (len(df) - len(labels_list))
#         temp_file = f"gofundme_with_labels_partial_{i}.csv"
#         df_temp.to_csv(temp_file, index=False)
#         print(f"✅ Saved intermediate results at {i} images: {temp_file}")


# df["image_labels"] = labels_list
# df.to_csv(OUTPUT_CSV, index=False)
# print(f"\n✅ Done! All images processed and saved to {OUTPUT_CSV}")


E0000 00:00:1760406719.786890 1998790 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
Processing Images:   0%|                       | 1/1000 [00:01<24:22,  1.46s/it]

[1/1000] Labels: ['Dog', 'Carnivores', 'Snout']...


Processing Images:   0%|                       | 2/1000 [00:04<34:55,  2.10s/it]

[2/1000] Labels: ['Dog', 'Jaw', 'Carnivores']...


Processing Images:   0%|                       | 3/1000 [00:06<38:40,  2.33s/it]

...

[998/1000] Labels: ['Cat', 'Felidae', 'Carnivores']...


Processing Images: 100%|████████████████████▉| 999/1000 [49:27<00:08,  8.28s/it]

[999/1000] Labels: ['Dog', 'Dog Clothes', 'Carnivores']...


Processing Images: 100%|████████████████████| 1000/1000 [49:30<00:00,  2.97s/it]

[1000/1000] Labels: ['Dog', 'Carnivores', 'Logo']...
✅ Saved intermediate results at 1000 images: gofundme_with_labels_partial_1000.csv

✅ Done! All images processed and saved to gofundme_with_labels.csv
